<a href="https://colab.research.google.com/github/Vibha111094/CSTestAutomation/blob/master/Abstractive_Summarization_with_BERT_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
!rm -rf cnn

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/device:GPU:0']


In [2]:
import sys

!test -d texar_repo || git clone https://github.com/asyml/texar.git texar_repo
if not 'texar_repo' in sys.path:
  sys.path += ['texar_repo']
!pip install funcsigs

In [0]:
tf.__version__

'1.13.1'

In [0]:
!tar -zxf cnn_stories.tgz

In [35]:
import string

# Display punctuation.

string_punctuation2=string.punctuation.replace('.', '')
print(string_punctuation2)

# for c in string.punctuation:
#     string.punctuation2

!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~


In [36]:
from os import listdir
import string
import nltk
nltk.download('punkt')

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a document into news story and highlights
def split_story(doc):
	# find first highlight
	index = doc.find('@highlight')
	# split into story and highlights
	story, highlights = doc[:index], doc[index:].split('@highlight')
	# strip extra white space around each highlight
	highlights = [h.strip() for h in highlights if len(h) > 0]
	return story, highlights

# load all stories in a directory
def load_stories(directory):
	stories = list()
	for name in listdir(directory):
		filename = directory + '/' + name
		# load document
		doc = load_doc(filename)
		# split into story and highlights
		story, highlights = split_story(doc)
		# store
		stories.append({'story':story, 'highlights':highlights})
	return stories



# def clean_lines(lines):
# 	cleaned = list()
# 	# prepare a translation table to remove punctuation
# 	table = str.maketrans('', '', string.punctuation)
# 	for line in lines:
# 		# strip source cnn office if it exists
# 		index = line.find('(CNN) -- ')
# 		if index > -1:
# 			line = line[index+len('(CNN)'):]
# 		# tokenize on white space
# 		#line = line.split()
# 		# convert to lower case
# 		#line = [word.lower() for word in line]
# 		# remove punctuation from each token
# 		#line = [w.translate(table) for w in line]
# 		line=line.lower()
# 		line=nltk.word_tokenize(line)
# 		# store as string
# 		cleaned.append(' '.join(line))
# 	# remove empty strings
# 	cleaned = [c for c in cleaned if len(c) > 0]
# 	return cleaned
#clean a list of lines

def clean_lines(lines):
	cleaned = list()
	# prepare a translation table to remove punctuation
	table = str.maketrans('', '', string_punctuation2)
	for line in lines:
		# strip source cnn office if it exists
		index = line.find('(CNN) -- ')
		if index > -1:
			line = line[index+len('(CNN)'):]
		# tokenize on white space
		line = line.split()
		# convert to lower case
		line = [word.lower() for word in line]
		# remove punctuation from each token
		line = [w.translate(table) for w in line]
		# remove tokens with numbers in them
		#line = [word for word in line if word.isalpha()]
		# store as string
		cleaned.append(' '.join(line))
	# remove empty strings
	cleaned = [c for c in cleaned if len(c) > 0]
	return cleaned


# load stories
directory = 'cnn/stories/'
stories = load_stories(directory)
print('Loaded Stories %d' % len(stories))

# clean stories
f1 = open("stories.txt",'w')
f2 = open("summary.txt",'w')
for example in stories:
  example['story'] = clean_lines(example['story'].split('\n'))
  example['highlights'] = clean_lines(example['highlights'])
  for i in example['highlights']:
    if(len(i)>399):
        print("The length is")
        print(len(i))
  f1.write(" ".join(example['story']))
  f1.write("\n")
  f2.write(" ".join(example['highlights']))
  f2.write("\n")
f1.close()
f2.close()
  
story = open("stories.txt").readlines()
summ = open("summary.txt").readlines() 
train_story = story[0:90000]
train_summ = summ[0:90000]

eval_story = story[90000:91579]
eval_summ = summ[90000:91579]

#print(eval_summ)


test_story = story[91579:92579]
test_summ = summ[91579:92579]


with open("train_story.txt",'w') as f:
  f.write("\n".join(train_story))
  
with open("train_summ.txt",'w') as f:
  f.write("\n".join(train_summ))
  
with open("eval_story.txt",'w') as f:
  f.write("\n".join(eval_story))
  
  
with open("eval_summ.txt",'w') as f:
  f.write("\n".join(eval_summ))
  
  
with open("test_story.txt",'w') as f:
  f.write("\n".join(test_story))
  
  
with open("test_summ.txt",'w') as f:
  f.write("\n".join(test_summ))  


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Loaded Stories 92579


# New Section

In [0]:
import os
import csv
import collections
import sys
from texar_repo.examples.bert.utils import data_utils, model_utils, tokenization
import importlib
import tensorflow as tf
import texar as tx 
from texar_repo.examples.bert import config_classifier as config_downstream
from texar_repo.texar.utils import transformer_utils
from texar_repo.examples.transformer.utils import data_utils, utils
from texar_repo.examples.transformer.bleu_tool import bleu_wrapper


In [38]:
#config

dcoder_config = {
    'dim': 768,
    'num_blocks': 6,
    'multihead_attention': {
        'num_heads': 8,
        'output_dim': 768
        # See documentation for more optional hyperparameters
    },
    'position_embedder_hparams': {
        'dim': 768
    },
    'initializer': {
        'type': 'variance_scaling_initializer',
        'kwargs': {
            'scale': 1.0,
            'mode': 'fan_avg',
            'distribution': 'uniform',
        },
    },
    'poswise_feedforward': tx.modules.default_transformer_poswise_net_hparams(
        output_dim=768)
}

loss_label_confidence = 0.9

random_seed = 1234
beam_width = -1
alpha = 0.6
hidden_dim = 768


opt = {
    'optimizer': {
        'type': 'AdamOptimizer',
        'kwargs': {
            'learning_rate':3e-4,
            'beta1': 0.9,
            'beta2': 0.997,
            'epsilon': 1e-10
        }
    }
}




lr = {
    'learning_rate_schedule': 'constant.linear_warmup.rsqrt_decay.rsqrt_depth',
    'lr_constant': 2 * (hidden_dim ** -0.5),
    'static_lr': 1e-3,
    'warmup_steps': 2000,
}

bos_token_id =101
eos_token_id = 102

model_dir= "./models"
run_mode= "train_and_evaluate"
batch_size = 32
test_batch_size = 32

max_train_epoch = 20
display_steps = 100
eval_steps = 100000

max_decoding_length = 400

max_seq_length_src = 512
max_seq_length_tgt = 400

bert_pretrain_dir = 'bert_pretrained_models/uncased_L-12_H-768_A-12'
print(bert_pretrain_dir)
#config

bert_pretrained_models/uncased_L-12_H-768_A-12


In [0]:
print(opt.get('kwargs'))

None


In [0]:
!mkdir bert_pretrained_models

In [7]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip -P bert_pretrained_models/;
!unzip bert_pretrained_models/uncased_L-12_H-768_A-12.zip -d bert_pretrained_models/


--2019-04-06 07:53:29--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.200.128, 2607:f8b0:4001:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘bert_pretrained_models/uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   139MB/s    in 2.8s    

2019-04-06 07:53:32 (139 MB/s) - ‘bert_pretrained_models/uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  bert_pretrained_models/uncased_L-12_H-768_A-12.zip
   creating: bert_pretrained_models/uncased_L-12_H-768_A-12/
  inflating: bert_pretrained_models/uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: bert_pretrained_models/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: bert_pretrained_models/uncased

In [0]:
class InputExample():
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence.
                For single sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second
                sequence. Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
                specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.src_txt = text_a
        self.tgt_txt = text_b
        
class InputFeatures():
    """A single set of features of data."""

    def __init__(self, src_input_ids,src_input_mask,src_segment_ids,tgt_input_ids,tgt_input_mask,tgt_labels):
        self.src_input_ids = src_input_ids
        self.src_input_mask = src_input_mask
        self.src_segment_ids = src_segment_ids
        self.tgt_input_ids = tgt_input_ids
        self.tgt_input_mask = tgt_input_mask 
        self.tgt_labels = tgt_labels
        
       
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for prediction."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with tf.gfile.Open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            i = 0
            for line in reader:
                lines.append(line)
        return lines


    @classmethod
    def _read_file(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with tf.gfile.Open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\n", quotechar=quotechar)
            lines = []
            i = 0
            for line in reader:
                lines.append(line)
        return lines
      
      
class CNNDailymail(DataProcessor):
    """Processor for the CoLA data set (GLUE version)."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_file(os.path.join(data_dir, "train_story.txt")),self._read_file(os.path.join(data_dir, "train_summ.txt")),
            "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_file(os.path.join(data_dir, "eval_story.txt")),self._read_file(os.path.join(data_dir, "eval_summ.txt")),
            "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_file(os.path.join(data_dir, "test_story.txt")),self._read_file(os.path.join(data_dir, "test_summ.txt")),
            "test")

    def _create_examples(self, src_lines,tgt_lines,set_type):
        examples = [] 
        for i,data in enumerate(zip(src_lines,tgt_lines)):
            guid = "%s-%s" % (set_type, i)
            if set_type == "test" and i == 0:
                continue
            else:
                #print(data)
                if len(data[0])==0 or len(data[1])==0:
                  continue
                src_lines = tokenization.convert_to_unicode(data[0][0])
                tgt_lines = tokenization.convert_to_unicode(data[1][0])
                examples.append(InputExample(guid=guid, text_a=src_lines,
                                         text_b=tgt_lines))
        return examples
  
  
def file_based_convert_examples_to_features(
        examples, max_seq_length_src,max_seq_length_tgt,tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)
    print("Vibha")

    for (ex_index, example) in enumerate(examples):
        #print("ex_index",ex_index)

        if (ex_index+1) %1000 == 0 :
          print("------------processed..{}...examples".format(ex_index))
          
        feature = convert_single_example(ex_index, example,
                                         max_seq_length_src,max_seq_length_tgt,tokenizer)

        def create_int_feature(values):
            return tf.train.Feature(
                int64_list=tf.train.Int64List(value=list(values)))

        features = collections.OrderedDict()
        features["src_input_ids"] = create_int_feature(feature.src_input_ids)
        features["src_input_mask"] = create_int_feature(feature.src_input_mask)
        features["src_segment_ids"] = create_int_feature(feature.src_segment_ids)

        features["tgt_input_ids"] = create_int_feature(feature.tgt_input_ids)
        features["tgt_input_mask"] = create_int_feature(feature.tgt_input_mask)
        features['tgt_labels'] = create_int_feature(feature.tgt_labels)
        
        
        
        #print(feature.tgt_labels)
        

        tf_example = tf.train.Example(
            features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())


def convert_single_example(ex_index, example, max_seq_length_src,max_seq_length_tgt,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""
    """
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i
    """
#    print(example)
#     tokens_a = tokenizer.tokenize(example['src_txt'])
#     tokens_b = tokenizer.tokenize(example['tgt_txt'])

    tokens_a = tokenizer.tokenize(example.src_txt)
    tokens_b = tokenizer.tokenize(example.tgt_txt)
    


    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    if len(tokens_a) > max_seq_length_src - 2:
            tokens_a = tokens_a[0:(max_seq_length_src - 2)]
    
    if len(tokens_b) > max_seq_length_tgt - 2:
            tokens_b = tokens_b[0:(max_seq_length_tgt - 2)]
    #print("yay")

    
    tokens_src = []
    segment_ids_src = []
    tokens_src.append("[CLS]")
    segment_ids_src.append(0)
    for token in tokens_a:
        tokens_src.append(token)
        segment_ids_src.append(0)
    tokens_src.append("[SEP]")
    segment_ids_src.append(0)
  

    tokens_tgt = []
    segment_ids_tgt = []
    tokens_tgt.append("[CLS]")
    #segment_ids_tgt.append(0)
    for token in tokens_b:
        tokens_tgt.append(token)
        #segment_ids_tgt.append(0)
    tokens_tgt.append("[SEP]")
    #segment_ids_tgt.append(0)

    input_ids_src = tokenizer.convert_tokens_to_ids(tokens_src)
   
    

    input_ids_tgt = tokenizer.convert_tokens_to_ids(tokens_tgt)
    
    #Adding begiining and end token
    input_ids_tgt = input_ids_tgt[:-1] 
    
    input_mask_src = [1] * len(input_ids_src)


    input_mask_tgt = [1] * len(input_ids_tgt)
    
    labels_tgt = input_ids_tgt[1:]
    
    
    labels_tgt.append(0)
    
    #print(len(input_ids_tgt))
    #print(len(input_mask_tgt))
    #print(len(labels_tgt))
    #print(len(segment_ids_tgt))
    
    while len(input_ids_src) < max_seq_length_src:
        input_ids_src.append(0)
        input_mask_src.append(0)
        segment_ids_src.append(0)

    while len(input_ids_tgt) < max_seq_length_tgt:
        input_ids_tgt.append(0)
        input_mask_tgt.append(0)
        segment_ids_tgt.append(0)
        labels_tgt.append(0)
    

    feature = InputFeatures( src_input_ids=input_ids_src,src_input_mask=input_mask_src,src_segment_ids=segment_ids_src,
        tgt_input_ids=input_ids_tgt,tgt_input_mask=input_mask_tgt,tgt_labels=labels_tgt)

    
    return feature


def file_based_input_fn_builder(input_file, max_seq_length_src,max_seq_length_tgt, is_training,
                                drop_remainder, is_distributed=False):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "src_input_ids": tf.FixedLenFeature([max_seq_length_src], tf.int64),
        "src_input_mask": tf.FixedLenFeature([max_seq_length_src], tf.int64),
        "src_segment_ids": tf.FixedLenFeature([max_seq_length_src], tf.int64),
        "tgt_input_ids": tf.FixedLenFeature([max_seq_length_tgt], tf.int64),
        "tgt_input_mask": tf.FixedLenFeature([max_seq_length_tgt], tf.int64),
        "tgt_labels" : tf.FixedLenFeature([max_seq_length_tgt], tf.int64),
        
        
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)
        print(example)
        print(example.keys())

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:

            if is_distributed:
                import horovod.tensorflow as hvd
                tf.logging.info('distributed mode is enabled.'
                                'size:{} rank:{}'.format(hvd.size(), hvd.rank()))
                # https://github.com/uber/horovod/issues/223
                d = d.shard(hvd.size(), hvd.rank())

                d = d.repeat()
                d = d.shuffle(buffer_size=100)
                d = d.apply(
                    tf.contrib.data.map_and_batch(
                        lambda record: _decode_record(record, name_to_features),
                        batch_size=batch_size//hvd.size(),
                        drop_remainder=drop_remainder))
            else:
                tf.logging.info('distributed mode is not enabled.')
                d = d.repeat()
                d = d.shuffle(buffer_size=100)
                d = d.apply(
                    tf.contrib.data.map_and_batch(
                        lambda record: _decode_record(record, name_to_features),
                        batch_size=batch_size,
                        drop_remainder=drop_remainder))

        else:
            d = d.apply(
                tf.contrib.data.map_and_batch(
                    lambda record: _decode_record(record, name_to_features),
                    batch_size=batch_size,
                    drop_remainder=drop_remainder))

        return d
    return input_fn
  
  
def get_dataset(processor,
                tokenizer,
                data_dir,
                max_seq_length_src,
                max_seq_length_tgt,
                batch_size,
                mode,
                output_dir,
                is_distributed=False):
    """
    Args:
        processor: Data Preprocessor, must have get_lables,
            get_train/dev/test/examples methods defined.
        tokenizer: The Sentence Tokenizer. Generally should be
            SentencePiece Model.
        data_dir: The input data directory.
        max_seq_length: Max sequence length.
        batch_size: mini-batch size.
        model: `train`, `eval` or `test`.
        output_dir: The directory to save the TFRecords in.
    """
    #label_list = processor.get_labels()
    if mode == 'train':
        #train_examples = processor.get_train_examples(data_dir)
        #train_file = os.path.join(output_dir, "train.tf_record")
        train_file = "gs://my_bert_summ/train2.tf_record"
        #file_based_convert_examples_to_features(
        #    train_examples, max_seq_length_src,max_seq_length_tgt,
        #    tokenizer, train_file)
        dataset = file_based_input_fn_builder(
            input_file=train_file,
            max_seq_length_src=max_seq_length_src,
            max_seq_length_tgt =max_seq_length_tgt,
            is_training=True,
            drop_remainder=True,
            is_distributed=is_distributed)({'batch_size': batch_size})
    elif mode == 'eval':
        #eval_examples = processor.get_dev_examples(data_dir)
        #eval_file = os.path.join(output_dir, "eval.tf_record")
        eval_file = "gs://my_bert_summ/eval2.tf_record"
        #file_based_convert_examples_to_features(
        #    eval_examples, max_seq_length_src,max_seq_length_tgt,
        #    tokenizer, eval_file)
        dataset = file_based_input_fn_builder(
            input_file=eval_file,
            max_seq_length_src=max_seq_length_src,
            max_seq_length_tgt =max_seq_length_tgt,
            is_training=False,
            drop_remainder=True,
            is_distributed=is_distributed)({'batch_size': batch_size})
    elif mode == 'test':
      
        #test_examples = processor.get_test_examples(data_dir)
        #test_file = os.path.join(output_dir, "predict.tf_record")
        test_file = "gs://my_bert_summ/predict2.tf_record"
        
        #file_based_convert_examples_to_features(
        #    test_examples, max_seq_length_src,max_seq_length_tgt,
        #    tokenizer, test_file)
        dataset = file_based_input_fn_builder(
            input_file=test_file,
            max_seq_length_src=max_seq_length_src,
            max_seq_length_tgt =max_seq_length_tgt,
            is_training=False,
            drop_remainder=True,
            is_distributed=is_distributed)({'batch_size': batch_size})
    return dataset

In [0]:
print(alpha)

0.6


In [6]:
bert_config = model_utils.transform_bert_to_texar_config(
            os.path.join(bert_pretrain_dir, 'bert_config.json'))
print(bert_config)



tokenizer = tokenization.FullTokenizer(
        vocab_file=os.path.join(bert_pretrain_dir, 'vocab.txt'),
        do_lower_case=True)

vocab_size = len(tokenizer.vocab)

processor = CNNDailymail()
train_dataset = get_dataset(processor,tokenizer,"./",max_seq_length_src,max_seq_length_tgt,4,'train',"./")
eval_dataset = get_dataset(processor,tokenizer,"./",max_seq_length_src,max_seq_length_tgt,4,'eval',"./")
test_dataset = get_dataset(processor,tokenizer,"./",max_seq_length_src,max_seq_length_tgt,4,'test',"./")
#del processor

{
  "embed": {
    "dim": 768,
    "name": "word_embeddings"
  },
  "encoder": {
    "dim": 768,
    "embedding_dropout": 0.1,
    "multihead_attention": {
      "dropout_rate": 0.1,
      "name": "self",
      "num_heads": 12,
      "num_units": 768,
      "output_dim": 768,
      "use_bias": true
    },
    "name": "encoder",
    "num_blocks": 12,
    "position_embedder_hparams": {
      "dim": 768
    },
    "position_embedder_type": "variables",
    "position_size": 512,
    "poswise_feedforward": {
      "layers": [
        {
          "kwargs": {
            "activation": "gelu",
            "name": "intermediate",
            "units": 3072,
            "use_bias": true
          },
          "type": "Dense"
        },
        {
          "kwargs": {
            "activation": null,
            "name": "output",
            "units": 768,
            "use_bias": true
          },
          "type": "Dense"
        }
      ]
    },
    "residual_dropout": 0.1,
    "use_bert_config": 

In [0]:
tf.size(train_dataset)

In [0]:
del processor

In [0]:
vocab_size

30522

In [7]:
#inputs to the model
src_input_ids = tf.placeholder(tf.int64, shape=(None, None))
src_segment_ids = tf.placeholder(tf.int64, shape=(None, None))
tgt_input_ids = tf.placeholder(tf.int64, shape=(None, None))
tgt_segment_ids = tf.placeholder(tf.int64, shape=(None, None))

batch_size = tf.shape(src_input_ids)[0]

src_input_length = tf.reduce_sum(1 - tf.to_int32(tf.equal(src_input_ids, 0)),
                             axis=1)
tgt_input_length = tf.reduce_sum(1 - tf.to_int32(tf.equal(src_input_ids, 0)),
                             axis=1)

labels = tf.placeholder(tf.int64, shape=(None, None))
is_target = tf.to_float(tf.not_equal(labels, 0))


global_step = tf.Variable(0, dtype=tf.int64, trainable=False)
learning_rate = tf.placeholder(tf.float64, shape=(), name='lr')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#create the iterator 
iterator = tx.data.FeedableDataIterator({
        'train': train_dataset, 'eval': eval_dataset, 'test': test_dataset})

batch = iterator.get_next()

In [9]:
#encoder Bert model
print("Intializing the Bert Encoder Graph")
with tf.variable_scope('bert'):
        embedder = tx.modules.WordEmbedder(
            vocab_size=bert_config.vocab_size,
            hparams=bert_config.embed)
        word_embeds = embedder(src_input_ids)

        # Creates segment embeddings for each type of tokens.
        segment_embedder = tx.modules.WordEmbedder(
            vocab_size=bert_config.type_vocab_size,
            hparams=bert_config.segment_embed)
        segment_embeds = segment_embedder(src_segment_ids)

        input_embeds = word_embeds + segment_embeds

        # The BERT model (a TransformerEncoder)
        #bert_config.encoder=tf.stop_gradient(bert_config.encoder)
        encoder = tx.modules.TransformerEncoder(hparams=bert_config.encoder)
        encoder_output = encoder(input_embeds, src_input_length)
        
        # Builds layers for downstream classification, which is also initialized
        # with BERT pre-trained checkpoint.
        with tf.variable_scope("pooler"):
            # Uses the projection of the 1st-step hidden vector of BERT output
            # as the representation of the sentence
            bert_sent_hidden = tf.squeeze(encoder_output[:, 0:1, :], axis=1)
            bert_sent_output = tf.layers.dense(
                bert_sent_hidden, config_downstream.hidden_dim,
                activation=tf.tanh)
            output = tf.layers.dropout(
                bert_sent_output, rate=0.1, training=tx.global_mode_train())


print("loading the bert pretrained weights")
# Loads pretrained BERT model parameters
init_checkpoint = os.path.join(bert_pretrain_dir, 'bert_model.ckpt')
init_checkpoint = "gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt"
model_utils.init_bert_checkpoint(init_checkpoint)

Intializing the Bert Encoder Graph
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
loading the bert pretrained weights


In [0]:
bert_config.encoder

In [0]:
type(encoder_output)

tensorflow.python.framework.ops.Tensor

In [0]:
encoder_output

<tf.Tensor 'bert/encoder_1/layer_11/ffn/LayerNorm/batchnorm/add_1:0' shape=(?, ?, 768) dtype=float32>

In [0]:
bert_sent_hidden

<tf.Tensor 'bert/pooler/Squeeze:0' shape=(?, 768) dtype=float32>

In [0]:
bert_sent_output

<tf.Tensor 'bert/pooler/dense/Tanh:0' shape=(?, 768) dtype=float32>

In [0]:
output

<tf.Tensor 'bert/pooler/dropout/cond/Merge:0' shape=(?, 768) dtype=float32>

In [0]:
#decoder part and mle losss
tgt_embedding = tf.concat(
    [tf.zeros(shape=[1, embedder.dim]), embedder.embedding[1:, :]], axis=0)

decoder = tx.modules.TransformerDecoder(embedding=tgt_embedding,
                             hparams=dcoder_config)
# For training
outputs = decoder(
    memory=encoder_output,
    memory_sequence_length=src_input_length,
    inputs=embedder(tgt_input_ids),
    sequence_length=tgt_input_length,
    decoding_strategy='train_greedy',
    mode=tf.estimator.ModeKeys.TRAIN
)
#print("output of the logits is")
#print(outputs.logits)
#output_word = tf.nn.softmax(outputs.logits)
# with tf.Session() as sess:
#     print("Yay!!Lion")
#     c=sess.run(output_word)
#     print(c)
#     sess.close()
mle_loss = transformer_utils.smoothing_cross_entropy(
        outputs.logits, labels, vocab_size, loss_label_confidence)
mle_loss = tf.reduce_sum(mle_loss * is_target) / tf.reduce_sum(is_target)

allvars = []
#print(len(allvars))
allvars = tf.contrib.framework.get_trainable_variables()
# print(allvars)
nonBert=[]
for v in allvars:
    v1=str(v)
    if "bert" not in v1:
        nonBert.append(v)
tvars =tf.trainable_variables()
non_bert_vars = [var for var in tvars if 'bert' not in var.name]

#print("nonBert is")        
#print(tvars)
    
train_op = tx.core.get_train_op(
mle_loss,
learning_rate=learning_rate,
variables=non_bert_vars,
global_step=global_step,
hparams=opt)



tf.summary.scalar('lr', learning_rate)
tf.summary.scalar('mle_loss', mle_loss)
summary_merged = tf.summary.merge_all()


In [0]:
opt.get()

In [0]:
print(non_bert_vars)

[<tf.Variable 'transformer_decoder/layer_0/beta:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/gamma:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/self_attention/multihead_attention/query/kernel:0' shape=(768, 512) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/self_attention/multihead_attention/key/kernel:0' shape=(768, 512) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/self_attention/multihead_attention/value/kernel:0' shape=(768, 512) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/self_attention/multihead_attention/output/kernel:0' shape=(512, 768) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/encdec_attention/beta:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/encdec_attention/gamma:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'transformer_decoder/layer_0/encdec_attention/multihead_attention/query/kernel:0' shape=(768, 512) 

In [0]:
tgt_embedding

<tf.Tensor 'concat:0' shape=(30522, 768) dtype=float32>

In [11]:
#prediction 
start_tokens = tf.fill([tx.utils.get_batch_size(src_input_ids)],
                       bos_token_id)
beam_width=4
predictions = decoder(
    memory=encoder_output,
    memory_sequence_length=src_input_length,
    decoding_strategy='infer_greedy',
    beam_width=beam_width,
    length_penalty=1,
    #alpha=alpha,
    start_tokens=start_tokens,
    end_token=eos_token_id,
    max_decoding_length=150,
    mode=tf.estimator.ModeKeys.PREDICT
)


print(beam_width)
if beam_width <= 1:
    inferred_ids = predictions[0].sample_id
else:
    # Uses the best sample by beam search
    inferred_ids = predictions['sample_id'][:, :, 0]
    


saver = tf.train.Saver(max_to_keep=5)
best_results = {'score': 0, 'epoch': -1}

4


In [0]:
encoder_output

<tf.Tensor 'bert/encoder_1/layer_11/ffn/LayerNorm/batchnorm/add_1:0' shape=(?, ?, 768) dtype=float32>

In [0]:
import os
def _train_epoch(sess, epoch, step, smry_writer):
        
            
        fetches = {
            'step': global_step,
            'train_op': train_op,
            'smry': summary_merged,
            'loss': mle_loss,
        }

        while True:
            try:
              feed_dict = {
                iterator.handle: iterator.get_handle(sess, 'train'),
                tx.global_mode(): tf.estimator.ModeKeys.TRAIN,
              }
              op = sess.run([batch],feed_dict)
              feed_dict = {
                   src_input_ids:op[0]['src_input_ids'],
                   src_segment_ids : op[0]['src_segment_ids'],
                   tgt_input_ids:op[0]['tgt_input_ids'],

                   labels:op[0]['tgt_labels'],
                   learning_rate: utils.get_lr(step, lr),
                   tx.global_mode(): tf.estimator.ModeKeys.TRAIN
                }


              fetches_ = sess.run(fetches, feed_dict=feed_dict)
              step, loss = fetches_['step'], fetches_['loss']
              if step and step % display_steps == 0:
                  with open(os.path.join(r'./gdrive/My Drive/models_log','model12_loss.txt'),'a')as  file_obj:
                      print(step,loss,file=file_obj)
                      #print(step+"loss:"+loss,file=file_obj)
                      #print("Step:"+step+"loss:"+loss,file=file_obj)
                      #print("sucess",file=file_obj)
                  logger.info('step: %d, loss: %.4f', step, loss)
                  print('step: %d, loss: %.4f' % (step, loss))
                  smry_writer.add_summary(fetches_['smry'], global_step=step)

              if step and step % 1000 == 0:
                  model_path = "gs://my_bert_summ/models12/model_"+str(step)+".ckpt"
                  #logger.info('saving model to %s', model_path)
                  print('saving model to %s' % model_path)
                  saver.save(sess, model_path)
                  _eval_epoch(sess, epoch,step,mode='eval')
              #if step and step % eval_steps == 0:
              #    _eval_epoch(sess, epoch, mode='eval')
            except tf.errors.OutOfRangeError:
                break

        return step

In [13]:
!pip install rouge

In [0]:
import json
from rouge import Rouge

In [0]:
def _eval_epoch(sess, epoch, step,mode):
        
        iterator.initialize_dataset(sess,'eval')
        avg = 0

        references, hypotheses,my_input,my_try = [], [], [], []
        bsize = test_batch_size
        fetches= {
                'inferred_ids': inferred_ids
            }
#         fetches2={
#             'p':p
#         }
#         print("The mystery")
#         print(p)
        bno=0
        while (bno<1):
            
            #print("Temp",temp)
            try:
              print("Batch",bno)
              feed_dict = {
              iterator.handle: iterator.get_handle(sess, 'eval'),
              tx.global_mode(): tf.estimator.ModeKeys.EVAL,
              }
              op = sess.run([batch],feed_dict)
              #print("The twist")
              #print(op[0]['src_input_ids'])
              
              
              feed_dict = {
                   src_input_ids:op[0]['src_input_ids'],
                   src_segment_ids : op[0]['src_segment_ids'],
                   tx.global_mode(): tf.estimator.ModeKeys.EVAL
              }
              fetches_= sess.run(fetches, feed_dict=feed_dict)

              
              labels = op[0]['tgt_labels']
             
              
              my_input.extend(m.tolist() for m in op[0]['src_input_ids'] )
              hypotheses.extend(h.tolist() for h in fetches_['inferred_ids'])
              references.extend(r.tolist() for r in labels)
             
#               inv_map = {v: k for k, v in tokenizer.vocab.items()}

#               for i in hypotheses:
#                   for j in i:
#                       print(j)
#                       if(inv_map[j][0]=='#'):
#                         inv_map[j].replace(" ##", "")
#                       #  " ". Join(inv_map[j]).replace(" ##", "")
#                       else:
#                         print(inv_map[j], end=" ")
#                   print("\n")
                 
#               for i in references:
#                   for j in i:
#                       if(inv_map[j][0]=='#'):
#                         inv_map[j].replace(" ##", "")
#                       else:
#                         print(inv_map[j], end=" ")
#                   print("\n")    
                
              hypotheses = utils.list_strip_eos(hypotheses, eos_token_id)
              references = utils.list_strip_eos(references, eos_token_id)
              references = utils.list_strip_eos(references, 0)
              my_input = utils.list_strip_eos(my_input, eos_token_id)
                
              rouge = Rouge()
              
              for i in range(1):
                  myindex = (4*bno)+i
 
                  hwords = tokenizer.convert_ids_to_tokens(hypotheses[myindex])
                  rwords = tokenizer.convert_ids_to_tokens(references[myindex])
                  iwords = tokenizer.convert_ids_to_tokens(my_input[myindex])
                  hwords = tx.utils.str_join(hwords).replace(" ##","")
                  rwords = tx.utils.str_join(rwords).replace(" ##","")
                  #print(len(rwords))
                  #rwords = tx.utils.str_join(rwords).replace("[PAD]","")
                  #rwords.replace("[PAD]", "")
                  #print(rwords)
                  #print(type(rwords))
                  #print(len(rwords))
                    
#                   iwords = tx.utils.str_join(iwords).replace(" ##","")
                  rouge = Rouge()
                  r = rouge.get_scores(hwords, rwords)
                  with open(os.path.join(r'./gdrive/My Drive/models_log','model12_rogue.txt'),'a')as  file_obj:
                      print("Rogue is:")
                      print(r,file=file_obj)
                      print("<original>",file=file_obj)
                      print(rwords,file=file_obj)
                      print("</original>",file=file_obj)
                      print("<generated>",file=file_obj)
                      print(hwords,file=file_obj)
                      print("</generated>",file=file_obj)
                      
                      
                  print("rougue is")
                  print(r)
#                   avg = avg+r[0]['rouge-1']['f']
#                   print("score is")
#                   print(avg)
#                  print("Original Paragraph",iwords)
#                  print("Original",rwords)
#                  print("Generated",hwords)
#                  logger2.info(hwords)
#              fname = "./tmp2.eval"
              

              
                        
             
              bno = bno+1
              
            except tf.errors.OutOfRangeError:
                break
                
#         hypotheses = tx.utils.str_join(hypotheses)
#         references = tx.utils.str_join(references)
#         hyp_fn, ref_fn = tx.utils.write_paired_text(
#         hypotheses, references, fname, mode='s')
#         rouge = Rouge()
#         r = rouge.get_scores(hyp_fn, ref_fn)
#         print
#         print("Rogue")
#         print(str(r))
#         eval_bleu = bleu_wrapper(ref_fn, hyp_fn, case_sensitive=True)
#         #eval_bleu = bleu_wrapper(ref_fn, ref_fn, case_sensitive=True)
#         eval_bleu = 100. * eval_bleu
#         print('rogue %.4f' % (r)) 
#        logger2.info('step:%d ,rogue %.4f',step,r)
#         if(step % 50000 == 0):
#             logger.info('epoch: %d, eval_bleu %.4f', epoch, eval_bleu)
#             print('epoch: %d, eval_bleu %.4f' % (epoch, eval_bleu))

#             if eval_bleu > best_results['score']:
#                 logger.info('epoch: %d, best bleu: %.4f', epoch, eval_bleu)
#                 best_results['score'] = eval_bleu
#                 best_results['epoch'] = epoch
#                 #model_path = os.path.join(model_dir, 'best-model.ckpt')
#                 model_path = "gs://my_bert_summ/models5/best-model.ckpt"
#                 logger.info('saving model to %s', model_path)
#                 print('saving model to %s' % model_path)
#                 saver.save(sess, model_path)
            
        
        


        if mode == 'hahaha':
            
            # Writes results to files to evaluate BLEU
            # For 'eval' mode, the BLEU is based on token ids (rather than
            # text tokens) and serves only as a surrogate metric to monitor
            # the training process
            #fname = os.path.join(model_dir, 'tmp.eval')
            fname = "./tmp.eval"
            #fname = "gs://my_bert_summ/models/tmp.eval"
            hypotheses = tx.utils.str_join(hypotheses)
            references = tx.utils.str_join(references)
            hyp_fn, ref_fn = tx.utils.write_paired_text(
                hypotheses, references, fname, mode='s')
            eval_bleu = bleu_wrapper(ref_fn, hyp_fn, case_sensitive=True)
            eval_bleu = 100. * eval_bleu
            logger.info('epoch: %d, eval_bleu %.4f', epoch, eval_bleu)
            print('epoch: %d, eval_bleu %.4f' % (epoch, eval_bleu))

            if eval_bleu > best_results['score']:
                logger.info('epoch: %d, best bleu: %.4f', epoch, eval_bleu)
                best_results['score'] = eval_bleu
                best_results['epoch'] = epoch
                #model_path = os.path.join(model_dir, 'best-model.ckpt')
                model_path = "gs://my_bert_summ/models10/best-model.ckpt"
                logger.info('saving model to %s', model_path)
                print('saving model to %s' % model_path)
                saver.save(sess, model_path)
               

In [0]:
def infer_single_example(story,actual_summary,tokenizer,sess):
      my_example = {"src_txt":story,
      "tgt_txt":actual_summary
      }
#      for key,val in my_example.items():
#          print (key)
      print(my_example['src_txt'])
      
   
      
      features = convert_single_example(1,my_example,max_seq_length_src,max_seq_length_tgt,tokenizer)
#      print(np.array(features.tgt_labels).reshape(1,-1))
      feed_dict = {
      src_input_ids:np.array(features.src_input_ids).reshape(1,-1),
      src_segment_ids:np.array(features.src_segment_ids).reshape(1,-1)
      }

      references, hypotheses = [], []
      fetches = {
      'inferred_ids': inferred_ids,
      }
      fetches_ = sess.run(fetches, feed_dict=feed_dict)
      labels = np.array(features.tgt_labels).reshape(1,-1)
      hypotheses.extend(h.tolist() for h in fetches_['inferred_ids'])
      references.extend(r.tolist() for r in labels)
      print("Hypotheses is")
      print(hypotheses)
      print("Reference is")
      print(references)
      hypotheses = utils.list_strip_eos(hypotheses, eos_token_id)
      references = utils.list_strip_eos(references, eos_token_id)
      hwords = tokenizer.convert_ids_to_tokens(hypotheses[0])
      rwords = tokenizer.convert_ids_to_tokens(references[0])

      hwords = tx.utils.str_join(hwords).replace(" ##","")
      rwords = tx.utils.str_join(rwords).replace(" ##","")
      print("Original",rwords)
      print("Generated",hwords)
      return hwords

In [0]:
import numpy as np
with tf.Session() as sess:
    model_dir = "gs://my_bert_summ/models10/"
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(model_dir))
    hwords = infer_single_example(story,summary,tokenizer,sess)
    print(hwords)


INFO:tensorflow:Restoring parameters from gs://my_bert_summ/models5/model_114000.ckpt
For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.
{'src_txt': 'For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.', 'tgt_txt': 'For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.Pope Francis said Sunday that he would hold a meeting of cardinals on February 14 during which I will name 15 new Cardinals who, coming from 13 countries from every continent, manifest the indissoluble links between the Church of Rome and the particular Churches present in the world, according to Vatican Radio. New cardinals are always important because they set the tone in t

In [0]:
summary ="For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.Pope Francis said Sunday that he would hold a meeting of cardinals on February 14 during which I will name 15 new Cardinals who, coming from 13 countries from every continent, manifest the indissoluble links between the Church of Rome and the particular Churches present in the world, according to Vatican Radio. New cardinals are always important because they set the tone in the church and also elect the next pope, CNN Senior Vatican Analyst John L. Allen said. They are sometimes referred to as the princes of the Catholic Church. The new cardinals come from countries such as Ethiopia, New Zealand and Myanmar."

In [0]:
story = str()
story = "For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.Pope Francis said Sunday that he would hold a meeting of cardinals on February 14 during which I will name 15 new Cardinals who, coming from 13 countries from every continent, manifest the indissoluble links between the Church of Rome and the particular Churches present in the world, according to Vatican Radio.New cardinals are always important because they set the tone in the church and also elect the next pope, CNN Senior Vatican Analyst John L. Allen said. They are sometimes referred to as the princes of the Catholic Church.The new cardinals come from countries such as Ethiopia, New Zealand and Myanmar.This is a pope who very much wants to reach out to people on the margins, and you clearly see that in this set, Allen said. You're talking about cardinals from typically overlooked places, like Cape Verde, the Pacific island of Tonga, Panama, Thailand, Uruguay.But for the second time since Francis' election, no Americans made the list.Francis pattern is very clear: He wants to go to the geographical peripheries rather than places that are already top-heavy with cardinals, Allen said.Christopher Bellitto, a professor of church history at Kean University in New Jersey, noted that Francis announced his new slate of cardinals on the Catholic Feast of the Epiphany, which commemorates the visit of the Magi to Jesus birthplace in Bethlehem.On feast of three wise men from far away, the Pope choices for cardinal say that every local church deserves a place at the big table.In other words, Francis wants a more decentralized church and wants to hear reform ideas from small communities that sit far from Catholicism power centers, Bellitto said.That does not mean Francis is the first pontiff to appoint cardinals from the developing world, though. Beginning in the 1920s, an increasing number of Latin American churchmen were named cardinals, and in the 1960s, St. John XXIII, whom Francis canonized last year, appointed the first cardinals from Japan, the Philippines and Africa.In addition to the 15 new cardinals Francis named on Sunday, five retired archbishops and bishops will also be honored as cardinals.Last year, Pope Francis appointed 19 new cardinals, including bishops from Haiti and Burkina Faso.CNNs Daniel Burke and Christabelle Fombu contributed to this report."
print(story)

For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.Pope Francis said Sunday that he would hold a meeting of cardinals on February 14 during which I will name 15 new Cardinals who, coming from 13 countries from every continent, manifest the indissoluble links between the Church of Rome and the particular Churches present in the world, according to Vatican Radio.New cardinals are always important because they set the tone in the church and also elect the next pope, CNN Senior Vatican Analyst John L. Allen said. They are sometimes referred to as the princes of the Catholic Church.The new cardinals come from countries such as Ethiopia, New Zealand and Myanmar.This is a pope who very much wants to reach out to people on the margins, and you clearly see that in this set, Allen said. You're talking about cardinals from typically overlooked places, like Cape Verde, the Pacific 

In [0]:
story ="For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world."

In [0]:
summary ="The 15 new cardinals will be installed on February 14.They come from countries such as Myanmar and Tonga.No Americans made the list this time or the previous time in Francis' papacy"

In [0]:
print(summary)

The 15 new cardinals will be installed on February 14.They come from countries such as Myanmar and Tonga.No Americans made the list this time or the previous time in Francis' papacy


In [0]:
tokenizer = tokenization.FullTokenizer(
      vocab_file=os.path.join(bert_pretrain_dir, 'vocab.txt'),
      do_lower_case=True)

In [0]:
summary = "For the second time during his papacy,Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.Pope Francis said Sunday that he would hold a meeting of cardinals on February 14 during which I will name 15 new Cardinals who, coming from 13 countries from every continent, manifest the indissoluble links between the Church of Rome and the particular Churches present in the world, according to Vatican Radio."

In [0]:
import numpy as np
with tf.Session() as sess:
    model_dir = "gs://my_bert_summ/models10/"
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(model_dir))
    hwords = infer_single_example(story,summary,tokenizer,sess)
    print(hwords)

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [39]:
#tx.utils.maybe_create_dir(model_dir)
#logging_file = os.path.join(model_dir, 'logging.txt')
model_dir = "gs://my_bert_summ/models12/"
my_info = "gs://my_bert_summ/info.txt"
logging_file= "logging.txt"
logging_file2="logging2.txt"
logger2 = utils.get_logger(logging_file2)
logger = utils.get_logger(logging_file)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    smry_writer = tf.summary.FileWriter(model_dir, graph=sess.graph)

    if run_mode == 'train_and_evaluate':
        logger.info('Begin running with train_and_evaluate mode')

        if tf.train.latest_checkpoint(model_dir) is not None:
            logger.info('Restore latest checkpoint in %s' % model_dir)
            saver.restore(sess, tf.train.latest_checkpoint(model_dir))
        
        iterator.initialize_dataset(sess)

        step = 5000
        for epoch in range(max_train_epoch):
          iterator.restart_dataset(sess, 'train')
          step = _train_epoch(sess, epoch, step, smry_writer)

    elif run_mode == 'test':
        
        logger.info('Begin running with test mode')

        logger.info('Restore latest checkpoint in %s' % model_dir)
        saver.restore(sess, tf.train.latest_checkpoint(model_dir))

        _eval_epoch(sess, 0,0, mode='eval')

    else:
        raise ValueError('Unknown mode: {}'.format(run_mode))

INFO:tensorflow:Restoring parameters from gs://my_bert_summ/models12/model_1000.ckpt
step: 1100, loss: 7.2520
step: 1200, loss: 7.0326
step: 1300, loss: 7.2196
step: 1400, loss: 6.8648
step: 1500, loss: 7.1534
step: 1600, loss: 7.3909
step: 1700, loss: 6.8392
step: 1800, loss: 7.3834
step: 1900, loss: 7.4793
step: 2000, loss: 7.3457
saving model to gs://my_bert_summ/models12/model_2000.ckpt
Batch 0
234
monsignor william lynn must remain in jail for now a judge rules lynns defense attorneys had filed a motion to release the 61yearold on house arrest a judge has delayed the decision on house arrest another hearing scheduled for july 5
<class 'str'>
234
Rogue is:
rougue is
[{'rouge-1': {'f': 0.05128204867850112, 'p': 0.16666666666666666, 'r': 0.030303030303030304}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.031117749564398823, 'p': 0.16666666666666666, 'r': 0.030303030303030304}}]
step: 2100, loss: 7.2346
step: 2200, loss: 7.8237
step: 2300, loss: 7.0058
step: 2400, los

KeyboardInterrupt: ignored

In [33]:
tf.__version__

'1.13.1'

In [0]:
orig_tokens = ["John", "Johanson", "'s",  "house"]
labels      = ["NNP",  "NNP",      "POS", "NN"]

### Output
bert_tokens = []

# Token map will be an int -> int mapping between the `orig_tokens` index and
# the `bert_tokens` index.
orig_to_tok_map = []

#tokenizer = tokenization.FullTokenizer(
#    vocab_file=vocab_file, do_lower_case=True)


tokenizer = tokenization.FullTokenizer(
        vocab_file=os.path.join(bert_pretrain_dir, 'vocab.txt'),
        do_lower_case=True)

bert_tokens.append("[CLS]")
for orig_token in orig_tokens:
  orig_to_tok_map.append(len(bert_tokens))
  bert_tokens.extend(tokenizer.tokenize(orig_token))
bert_tokens.append("[SEP]")

print(bert_tokens)

['[CLS]', 'john', 'johan', '##son', "'", 's', 'house', '[SEP]']


In [0]:
print(tokenizer.vocab.items())

In [0]:
inv_map = {v: k for k, v in tokenizer.vocab.items()}

In [27]:
print(inv_map[0])

[PAD]


In [0]:
tokenizer.convert_ids_to_tokens([102])

['[SEP]']